Distribution of race and gender by:

1. year
2. R1 vs other
3. public, private control
4. Region
5. HBCU
6. Tribal
7. HSI
8. MSI
9. Womens' college
10. Selectivity index

three metrics:

1. Proportion
2. Relative over/under representation by race and gender, given the general prop. of that race and gender on that given year
3. Ratio with respect to the expected proportion, given the especiality distribution of the group of institution in the combination of 2-10

In [45]:
%config Completer.use_jedi = False

In [46]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import numpy as np
import re
import string

from libs.LastNamesInference import LastNamesInference

In [47]:
address_clean = pd.read_pickle('/data/datasets/WOS/US/address_clean.p')

In [48]:
address_clean.hsi.value_counts(normalize=True)

not HSI    0.957125
HSI        0.042875
Name: hsi, dtype: float64

In [49]:
df_papers = pd.read_pickle('/data/datasets/WOS/US/US_papers.p')

- RQ2: In the present, are marginalized R&G more concentrated on a specific institution type?
- RQ3: If we normalized participation by fields, does RQ2 change?


In [50]:
howard = address_clean[address_clean.carnegie_id == '131520']

In [51]:
howard

,cluster_id,id_art,grid_id,carnegie_id,carnegie_name,r1,control,obereg,hbcu,tribal,...,Top1CR_Q,npapers,impact,avg_citations,npapers_Q,impact_Q,avg_citations_Q,usnr_rank,usnr_rank_cat,avg_citations_Q10
14974875,7282087,44152683,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
14974876,42542991,52231209,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
14974877,51575029,63672355,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
14974878,27224778,47822200,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
14974879,26103310,46241265,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14984638,50163063,62984663,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
14984639,50917584,51562409,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
14984640,5917671,29810409,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"
14984641,18597559,61457782,grid.257127.4,131520,Howard University,not R1,private_nfp,Mid_East,HBCU,not tribal,...,"(0.0, 0.02)",11596,13475.148,1.162039,"(5, 64564)","(0.85, 95992.29)","(0.1, 1.47)",83.0,top_100,"(0.1, 1.24)"


In [52]:
address_clean[address_clean.carnegie_id == '166027']

,cluster_id,id_art,grid_id,carnegie_id,carnegie_name,r1,control,obereg,hbcu,tribal,...,Top1CR_Q,npapers,impact,avg_citations,npapers_Q,impact_Q,avg_citations_Q,usnr_rank,usnr_rank_cat,avg_citations_Q10
2976819,2334755,29818433,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
2976820,53798920,45131934,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
2976821,40300552,48093667,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
2976822,49791729,31774021,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
2976823,32092575,54343911,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697823,9855205,62124190,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
3697824,50135279,56168647,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
3697825,48246028,55077141,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"
3697826,52446264,47840603,grid.38142.3c,166027,Harvard University,R1,private_nfp,New_England,not HBCU,not tribal,...,"(0.03, 0.17)",445620,969871.123,2.176453,"(146327, 445620)","(246486.01, 969871.12)","(1.77, 4.07)",2.0,top_10,"(2.17, 4.07)"


In [53]:
address_clean.carnegie_id.value_counts()

166027    721009
162928    327219
170976    314933
190415    306044
110662    303932
           ...  
120768         2
108807         2
123527         1
149532         1
121886         1
Name: carnegie_id, Length: 685, dtype: int64

In [54]:
address_clean.r1.value_counts(normalize=True)

R1        0.837239
not R1    0.162761
Name: r1, dtype: float64

In [55]:
def split_by_gender(df_race):
    df_race = df_race.copy()
    df_race['gender'] = df_race.gender.str.upper()

    boolean_mask_M = df_race.gender == 'M'
    boolean_mask_F = df_race.gender == 'F'

    race_dist_M = df_race.loc[boolean_mask_M,['white','hispanic','black','asian']]
    race_dist_F = df_race.loc[boolean_mask_F,['white','hispanic','black','asian']]

    race_dist_M.columns = race_dist_M.columns + '_M'
    race_dist_F.columns = race_dist_F.columns + '_F'
    race_dist_MF = pd.concat([race_dist_M,race_dist_F]).fillna(0).sort_index()
    df_race = df_race.join(race_dist_MF,how='inner' )
    return df_race


In [56]:
def infer_race(us_papers, df=None):
    papers = us_papers.drop_duplicates('cluster_ID')
    lni = LastNamesInference(names = papers.nom)
    if df is None:
        df = us_papers.copy()
    tqdm.pandas(desc="inferring race from lastnames")
    lastname_race_dist = df.progress_apply(lambda x: lni.get_name_dist(lastname=x.nom), axis=1)
    df[lni.prob_order] = pd.DataFrame(lastname_race_dist.to_list(), index=df.index)
    df = split_by_gender(df)
    return df

In [57]:
df = df_papers.merge(address_clean, how='inner', left_on=['cluster_ID','id_art'], right_on=['cluster_id', 'id_art'])

In [58]:
df.Annee_Bibliographique.value_counts()

2018    1735609
2016    1667558
2017    1667092
2015    1602143
2014    1550742
2013    1505349
2012    1434661
2011    1275937
2010    1146411
2009    1080101
2008    1041258
2019     326573
Name: Annee_Bibliographique, dtype: int64

In [59]:
# I remove articles from 2019, due to small sample size
df = df[df.Annee_Bibliographique<2019]

In [60]:
df_race = infer_race(df)

imputing by the mean: 100% 3400405/3400405 [00:12<00:00, 280727.21it/s]


inferring race from lastnames:   0%|          | 0/15706861 [00:00<?, ?it/s]

In [61]:
df_race.tribal.value_counts()

not tribal    11176882
Name: tribal, dtype: int64

I am removing the 'tribal' flag because we are currently unable to find any article from tribal colleges

In [62]:
df_authors = df_race[['cluster_ID','Annee_Bibliographique','Prenom','nom','carnegie_id', #ID and wos metadata
         'carnegie_name', 'r1', 'control', 'obereg', 'hbcu', 'hsi', 'msi', 'womens', 'selindex', #Carnegie groups
         'usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q','avg_citations_Q10', # citations groups
         'white', 'hispanic','black', 'asian', 'white_M', 'hispanic_M', 'black_M', 'asian_M', # race and gender
       'white_F', 'hispanic_F', 'black_F', 'asian_F']]

In [63]:
df_authors = df_authors.drop_duplicates() 

### Methodological note:

I'm keeping single instances of authors institution, etc. But I will do double counting of authors for multiple institutions/disciplines/specialities

In [64]:
df_authors.cluster_ID.value_counts()

19891624    84
11459727    82
25654258    76
50594731    70
35507949    70
            ..
20813996     1
23253880     1
23253719     1
23253369     1
55571533     1
Name: cluster_ID, Length: 2568916, dtype: int64

- RQ1: How is the proportion of authors by group evolving over time? Is the same tendency for all institutions’ type?

In [ ]:
prop_df = df_authors.groupby(['Annee_Bibliographique','r1', 'control', 'obereg', 'hbcu', 'hsi', 'msi', 'womens', 'selindex',
                              'usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q','avg_citations_Q10'])\
                [['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']].mean()
size = df_authors.groupby(['Annee_Bibliographique','r1', 'control', 'obereg', 'hbcu', 'hsi', 'msi', 'womens', 'selindex',
                          'usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q','avg_citations_Q10'])['cluster_ID'].count()

In [ ]:
1 

In [ ]:
prop_df = prop_df.join(size).reset_index().rename(columns={'cluster_ID':'n'})

In [ ]:
prop_df = prop_df[prop_df.n>100] # I exclude cases with low sample size

### relative difference// relative over-under representation: 

is the relative under/over representation of a group, given its mean proportion that given year

In [ ]:
mean_dist = df_authors.groupby('Annee_Bibliographique')[['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']].mean().reset_index()

In [ ]:
rel_diff = prop_df.copy()

In [ ]:
mean_dist.columns = ['Annee_Bibliographique', 'avg_white_M', 'avg_hispanic_M', 'avg_black_M', 'avg_asian_M','avg_white_F', 'avg_hispanic_F', 'avg_black_F', 'avg_asian_F']

In [ ]:
rel_diff = rel_diff.merge(mean_dist, on='Annee_Bibliographique')

In [ ]:
rel_diff[['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']] = \
rel_diff[['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']]/\
rel_diff[['avg_white_M', 'avg_hispanic_M', 'avg_black_M', 'avg_asian_M','avg_white_F', 'avg_hispanic_F', 'avg_black_F', 'avg_asian_F']].to_numpy()

In [ ]:
rel_diff = rel_diff[['Annee_Bibliographique', 'r1', 'control', 'obereg', 'hbcu',
       'hsi', 'msi', 'womens', 'selindex','usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q','avg_citations_Q10',
                     'white_M', 'hispanic_M', 'black_M','asian_M', 'white_F', 'hispanic_F', 'black_F', 'asian_F', 'n']]

In [ ]:
prop_df.to_csv('../../results/institutions/rg_carnegie.csv', index=False)
rel_diff.to_csv('../../results/institutions/rg_carnegie_rel_diff.csv', index=False)

# distribution respect expected by disciplines
## There are 294394 combinations of categories. This code is too computationally expensive


In [ ]:
# prop_df= pd.read_csv('../../results/institutions/rg_carnegie.csv')


# speciality_rg_dist = df_authors.groupby(['Annee_Bibliographique','ESpecialite'])[['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']].mean()

# def wm(x, cols, w="n"):
#     return pd.Series(np.average(x[cols], weights=x[w], axis=0), cols)

# def prop_df_esp_control_cat(cat):
#     univ_cat_specialy_dist = df_authors.groupby(['Annee_Bibliographique','ESpecialite',cat])[['cluster_ID']].count().reset_index()
#     univ_cat_specialy_dist = univ_cat_specialy_dist.rename(columns={'cluster_ID':'n'})

#     univ_cat_specialy_dist = univ_cat_specialy_dist.merge(speciality_rg_dist, on=['Annee_Bibliographique','ESpecialite'])
#     univ_cat_specialy_dist = univ_cat_specialy_dist[univ_cat_specialy_dist.n>=100] #I exclude cases with low sample size

#     expected_dist = univ_cat_specialy_dist.groupby(['Annee_Bibliographique', cat]).apply(wm, ['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F'])
#     exp_size = univ_cat_specialy_dist.groupby(['Annee_Bibliographique', cat]).n.sum()

#     expected_dist = expected_dist.join(exp_size).reset_index().rename(columns={'cluster_ID':'n'})
#     expected_dist = expected_dist[expected_dist.n>=100] #I exclude cases with low sample size
#     expected_dist[cat] = expected_dist[cat].astype(str)

#     expected_dist = expected_dist.set_index(['Annee_Bibliographique', cat])

#     prop_df_ = prop_df.groupby(['Annee_Bibliographique', cat]).apply(wm, ['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F'])
#     cat_n =  prop_df.groupby(['Annee_Bibliographique',cat]).n.sum()
#     prop_df_['n'] = cat_n
#     prop_df_,expected_dist = prop_df_.align(expected_dist,axis=0,join='inner')
#     prop_df_esp_control = prop_df_/expected_dist
#     prop_df_esp_control = prop_df_esp_control.reset_index()
#     prop_df_esp_control = prop_df_esp_control.drop(columns=['n'])
#     prop_df_esp_control = prop_df_esp_control.merge(cat_n.reset_index(), left_on = ['Annee_Bibliographique',cat], right_on=['Annee_Bibliographique',cat])
#     prop_df_esp_control = prop_df_esp_control.rename(columns={cat:'institution_cat'})
#     prop_df_esp_control['institution_cat_group'] = cat
#     return (prop_df_esp_control)

# univ_cats = ['r1', 'control', 'obereg', 'hbcu', 'hsi', 'msi', 'womens', 'selindex','usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q']

# prop_df_esp_control = pd.DataFrame()
# for cat in tqdm(univ_cats):
#     prop_df_esp_control_ = prop_df_esp_control_cat(cat)
#     prop_df_esp_control = pd.concat([prop_df_esp_control,prop_df_esp_control_])

# prop_df_esp_control.to_csv('../../results/institutions/rg_carnegie_disp_control.csv', index=False)

## authors with more than 1 paper

In [ ]:
authors_n = df_authors.cluster_ID.value_counts()

In [ ]:
df_authors_2 = df_authors[df_authors.cluster_ID.isin(authors_n[authors_n>1].index)].reset_index()

In [ ]:
prop_df_2 = df_authors_2.groupby(['Annee_Bibliographique','r1', 'control', 'obereg', 'hbcu', 'hsi', 'msi', 'womens', 'selindex',
                              'usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q','avg_citations_Q10'])\
                [['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']].mean()
size_2 = df_authors_2.groupby(['Annee_Bibliographique','r1', 'control', 'obereg', 'hbcu', 'hsi', 'msi', 'womens', 'selindex',
                          'usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q','avg_citations_Q10'])['cluster_ID'].count()

In [ ]:
prop_df_2 = prop_df_2.join(size_2).reset_index().rename(columns={'cluster_ID':'n'})

In [ ]:
prop_df_2 = prop_df_2[prop_df_2.n>100] # I exclude cases with low sample size

### relative difference// relative over-under representation: 

is the relative under/over representation of a group, given its mean proportion that given year

In [ ]:
mean_dist_2 = df_authors_2.groupby('Annee_Bibliographique')[['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']].mean().reset_index()

In [ ]:
rel_diff_2 = prop_df_2.copy()

In [ ]:
mean_dist_2.columns = ['Annee_Bibliographique', 'avg_white_M', 'avg_hispanic_M', 'avg_black_M', 'avg_asian_M','avg_white_F', 'avg_hispanic_F', 'avg_black_F', 'avg_asian_F']

In [ ]:
rel_diff_2 = rel_diff_2.merge(mean_dist_2, on='Annee_Bibliographique')

In [ ]:
rel_diff_2[['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']] = \
rel_diff_2[['white_M', 'hispanic_M', 'black_M', 'asian_M','white_F', 'hispanic_F', 'black_F', 'asian_F']]/\
rel_diff_2[['avg_white_M', 'avg_hispanic_M', 'avg_black_M', 'avg_asian_M','avg_white_F', 'avg_hispanic_F', 'avg_black_F', 'avg_asian_F']].to_numpy()

In [ ]:
rel_diff_2 = rel_diff_2[['Annee_Bibliographique', 'r1', 'control', 'obereg', 'hbcu',
       'hsi', 'msi', 'womens', 'selindex','usnr_rank_cat','Top10CR_Q','Top5CR_Q','Top1CR_Q','impact_Q','avg_citations_Q','avg_citations_Q10',
                     'white_M', 'hispanic_M', 'black_M','asian_M', 'white_F', 'hispanic_F', 'black_F', 'asian_F', 'n']]

In [ ]:
prop_df_2.to_csv('../../results/institutions/rg_carnegie_2.csv', index=False)
rel_diff_2.to_csv('../../results/institutions/rg_carnegie_rel_diff_2.csv', index=False)